In [1]:
import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import timeit

from tqdm.notebook import tqdm

In [2]:
stats_keys = [
    "accuracy",
    "p_iwl",
    "context contains query class",
    "ic_pred",
    "iw_pred",
    "num p_iwl >= 0.5",
    "p_iwl given context contains query class",
    "loss"
    # "similarity",
]

# Transformer
stats_keys = [
    "accuracy",
    "context contains query class",
    "loss"
]

In [3]:
# repo_path = "/Users/chanb/research/ualberta/icl/simple_icl"
# results_dir = "/Users/chanb/research/ualberta/icl/scratch/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/noisy_labels_0.01/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/paper_experiments/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/simple_icl/experiments/test_convergence_eval"

repo_path = "/Users/chanb/research/ualberta/icl/simple_icl"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/paper_experiments/evaluation_results"
results_dir = "/Users/chanb/research/ualberta/icl/cc_results/paper_experiments/rebuttal/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/paper_experiments/evaluation_results-e2e"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/paper_experiments/evaluation_results-heldout_features"

# repo_path = "/home/chanb/src/simple_icl"
# results_dir = "/home/chanb/scratch/simple_icl/results"

# repo_path = "/home/bryanpu1/projects/icl/simple_icl"
# results_dir = "/home/bryanpu1/projects/icl/scratch/evaluation_results"
# results_dir = "/home/bryanpu1/projects/icl/cc_results/evaluation_results"

In [4]:
# variant_name = "synthetic-simple_icl"
# variant_name = "synthetic-transformer"
# variant_name = "synthetic-transformer-context_len"
# variant_name = "synthetic-transformer-large_num_low_freq"
# variant_name = "synthetic-transformer-large_num_low_freq-input_noise_0.2"
# variant_name = "synthetic-transformer-no_noise"
# variant_name = "synthetic-transformer-noisy_inputs_0.2"
# variant_name = "synthetic-transformer-noisy_labels_0.01"
# variant_name = "synthetic-transformer-noisy_labels_0.1"
# variant_name = "synthetic-transformer-num_relevant_contexts"

# variant_name = "omniglot-input_noise-larger_bs"
# variant_name = "omniglot-input_noise"
# variant_name = "omniglot-p_high"
# variant_name = "omniglot-p_relevant"
# variant_name = "omniglot-num_contexts"
# variant_name = "omniglot-num_relevant_contexts"

# variant_name = "synthetic-transformer-noisy_inputs"
# variant_name = "synthetic-transformer-noisy_labels"
# variant_name = "synthetic-transformer-num_contexts"
# variant_name = "synthetic-transformer-num_low_freq"
# variant_name = "synthetic-transformer-num_relevant_contexts"
# variant_name = "synthetic-transformer-input_noise-longer_epoch"
# variant_name = "synthetic-transformer-p_high"
# variant_name = "synthetic-transformer-p_relevant"

# Post submission

# variant_name = "synthetic-simple_icl-p_high"
# variant_name = "synthetic-simple_icl-num_contexts"
# variant_name = "synthetic-simple_icl-num_low_freq"
# variant_name = "synthetic-e2e_alpha-num_low_freq"
# variant_name = "synthetic-simple_icl-noisy_inputs"

# variant_name = "synthetic-gru-noisy_inputs"

# variant_name = "synthetic-transformer-4_tf_blocks-noisy_inputs"
# variant_name = "synthetic-transformer-8_tf_blocks-noisy_inputs"
variant_name = "binary_synthetic-transformer-num_low_freq"

os.makedirs(
    os.path.join(repo_path, "plot_utils/plots/agg_stats"),
    exist_ok=True
)

In [5]:
stats_file = os.path.join(repo_path, "plot_utils/plots/agg_stats", "{}{}.feather".format(variant_name, "-heldout" if "heldout" in results_dir else ""))

NUM_PARALLEL = 2

minibatch_count = 0
minibatch_content = ""
if os.path.isfile(stats_file):
    stats = pd.read_feather(stats_file)
else:
    stats = []

    if variant_name.startswith("synthetic-simple_icl"):
        all_dirs = [
            variant_name.replace("synthetic-simple_icl", "synthetic-alpha"),
            # variant_name.replace("synthetic-simple_icl", "synthetic-ic_predictor"),
            # variant_name.replace("synthetic-simple_icl", "synthetic-iw_predictor"),
            variant_name.replace("synthetic-simple_icl", "synthetic-ic"),
            variant_name.replace("synthetic-simple_icl", "synthetic-iw"),
        ]
    else:
        all_dirs = [variant_name]

    for curr_dir in all_dirs:
        curr_results_dir = os.path.join(results_dir, curr_dir)
        if repo_path.startswith("/Users"):
            it = tqdm(os.listdir(curr_results_dir))
        else:
            # it = os.listdir(curr_results_dir)
            it = tqdm(os.listdir(curr_results_dir))

        for run_name in it:
            tic = timeit.default_timer()

            if os.path.isdir(os.path.join(curr_results_dir, run_name)) and run_name.startswith("p_relevant_context"):
                # IC/IW
                for baseline_name in os.listdir(os.path.join(curr_results_dir, run_name)):
                    prefix = "-".join(baseline_name.split("-")[:-9])
                    seed = baseline_name.split("-")[-9]

                    data = pickle.load(open(os.path.join(curr_results_dir, run_name, baseline_name), "rb"))
                    for eval_name in data["stats"]:
                        for stats_key in stats_keys:
                            if not stats_key in data["stats"][eval_name]:
                                continue

                            factor = 1
                            if stats_key == "accuracy":
                                factor = 1 / 100

                            stats.append(
                                dict(
                                    model_type="iw" if "p_relevant_context_0.0" in baseline_name else "ic",
                                    dirname=curr_dir,
                                    variant=prefix,
                                    seed=seed,
                                    eval_name=eval_name,
                                    stats_key=stats_key,
                                    stats=[float(val) * factor for val in data["stats"][eval_name][stats_key]],
                                    **{
                                        "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix.split("-")
                                    },
                                )
                            )
                            stats[-1]["p_relevant_context"] = run_name.split("_")[-1]
                            # if "p_relevant_context_1.0" in baseline_name:
                            #     stats.append(
                            #         dict(
                            #             model_type="transformer",
                            #             dirname=curr_dir,
                            #             variant=prefix,
                            #             seed=seed,
                            #             eval_name=eval_name,
                            #             stats_key=stats_key,
                            #             stats=[float(val) * factor for val in data["stats"][eval_name][stats_key]],
                            #             **{
                            #                 "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix.split("-")
                            #             },
                            #         )
                            #     )
                            # stats[-1]["p_relevant_context"] = run_name.split("_")[-1]

            else:
                # Transformer
                if "p_relevant_context_0.0" in run_name or "p_relevant_context_1.0" in run_name:
                    continue

                prefix = "-".join(run_name.split("-")[:-9])
                seed = run_name.split("-")[-9]

                data = pickle.load(open(os.path.join(curr_results_dir, run_name), "rb"))

                training_info_file = os.path.join(os.path.join(results_dir, os.pardir), "training_info", variant_name, run_name)
                batches = None
                get_training_info = True
                if os.path.isfile(training_info_file):
                    try:
                        batches = pickle.load(open(training_info_file, "rb"))
                        stats.append(
                            dict(
                                dirname=curr_dir,
                                variant=prefix,
                                seed=seed,
                                **{
                                    "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix.split("-")
                                },
                                batches={key: [i[key].tolist() for i in batches] for key in batches[0]},
                                run_name=run_name,
                            )
                        )
                        get_training_info = False
                    except:
                        print("{} failed to load".format(training_info_file))

                if get_training_info:
                    minibatch_count += 1
                    minibatch_content += "JAX_PLATFORMS=cpu python {}/plot_utils/simulate_minibatches.py --evaluation_file={} --repo_path={} --results_dir={} &\n".format(
                        repo_path,
                        os.path.join(curr_results_dir, run_name),
                        repo_path,
                        results_dir,
                    )
                    if minibatch_count % NUM_PARALLEL == 0:
                        minibatch_content += "wait \n"

                for eval_name in data["stats"]:
                    for stats_key in stats_keys:
                        if not stats_key in data["stats"][eval_name]:
                            continue

                        factor = 1
                        if stats_key == "accuracy":
                            factor = 1 / 100

                        prefix = prefix.replace("in_weight", "iw")
                        prefix = prefix.replace("in_context", "ic")

                        stats.append(
                            dict(
                                model_type="transformer" ,
                                dirname=curr_dir,
                                variant=prefix,
                                seed=seed,
                                eval_name=eval_name,
                                stats_key=stats_key,
                                stats=[float(val) * factor for val in data["stats"][eval_name][stats_key]],
                                run_name=run_name,
                                **{
                                    "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix.split("-")
                                },
                            )
                        )

            toc = timeit.default_timer()
    stats = pd.DataFrame(stats)
    stats.to_feather(stats_file)
    minibatch_content += "wait \n"

# with open("generate_fake_minibatches-{}{}.sh".format(variant_name, "-heldout" if "heldout" in variant_name else ""), "w") as f:
#     f.writelines(minibatch_content)

  0%|          | 0/63 [00:00<?, ?it/s]

In [6]:
stats

,model_type,dirname,variant,seed,eval_name,stats_key,stats,run_name,dataset_size,conditioning,num_low_freq_classes
0,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_4096-conditioning_iw-num_low_freq...,seed_2,eval-none-flip_label,accuracy,"[0.51, 0.05, 0.047, 0.055, 0.05800000000000001...",dataset_size_4096-conditioning_in_weight-num_l...,4096,iw,500
1,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_4096-conditioning_iw-num_low_freq...,seed_2,eval-none-flip_label,context contains query class,"[0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.9...",dataset_size_4096-conditioning_in_weight-num_l...,4096,iw,500
2,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_4096-conditioning_iw-num_low_freq...,seed_2,eval-none-flip_label,loss,"[0.8023144006729126, 5.15637731552124, 5.27727...",dataset_size_4096-conditioning_in_weight-num_l...,4096,iw,500
3,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_4096-conditioning_iw-num_low_freq...,seed_2,eval-none,accuracy,"[0.533, 0.944, 0.946, 0.941, 0.937, 0.94000000...",dataset_size_4096-conditioning_in_weight-num_l...,4096,iw,500
4,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_4096-conditioning_iw-num_low_freq...,seed_2,eval-none,context contains query class,"[0.946, 0.946, 0.946, 0.946, 0.946, 0.946, 0.9...",dataset_size_4096-conditioning_in_weight-num_l...,4096,iw,500
...,...,...,...,...,...,...,...,...,...,...,...
1318,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_262144-conditioning_iw-num_low_fr...,seed_2,eval-iw,context contains query class,"[0.942, 0.942, 0.942, 0.942, 0.942, 0.942, 0.9...",dataset_size_262144-conditioning_in_weight-num...,262144,iw,500
1319,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_262144-conditioning_iw-num_low_fr...,seed_2,eval-iw,loss,"[0.7783243656158447, 0.05321662500500679, 0.05...",dataset_size_262144-conditioning_in_weight-num...,262144,iw,500
1320,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_262144-conditioning_iw-num_low_fr...,seed_2,pretraining,accuracy,"[0.502, 0.9902, 0.9902, 0.9902, 0.9902, 0.9902...",dataset_size_262144-conditioning_in_weight-num...,262144,iw,500
1321,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_262144-conditioning_iw-num_low_fr...,seed_2,pretraining,context contains query class,"[0.9435, 0.9435, 0.9435, 0.9435, 0.9435, 0.943...",dataset_size_262144-conditioning_in_weight-num...,262144,iw,500


In [7]:
len(stats)

1323

In [8]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   model_type            1323 non-null   object
 1   dirname               1323 non-null   object
 2   variant               1323 non-null   object
 3   seed                  1323 non-null   object
 4   eval_name             1323 non-null   object
 5   stats_key             1323 non-null   object
 6   stats                 1323 non-null   object
 7   run_name              1323 non-null   object
 8   dataset_size          1323 non-null   object
 9   conditioning          1323 non-null   object
 10  num_low_freq_classes  1323 non-null   object
dtypes: object(11)
memory usage: 113.8+ KB


In [9]:
stats

,model_type,dirname,variant,seed,eval_name,stats_key,stats,run_name,dataset_size,conditioning,num_low_freq_classes
0,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_4096-conditioning_iw-num_low_freq...,seed_2,eval-none-flip_label,accuracy,"[0.51, 0.05, 0.047, 0.055, 0.05800000000000001...",dataset_size_4096-conditioning_in_weight-num_l...,4096,iw,500
1,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_4096-conditioning_iw-num_low_freq...,seed_2,eval-none-flip_label,context contains query class,"[0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.9...",dataset_size_4096-conditioning_in_weight-num_l...,4096,iw,500
2,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_4096-conditioning_iw-num_low_freq...,seed_2,eval-none-flip_label,loss,"[0.8023144006729126, 5.15637731552124, 5.27727...",dataset_size_4096-conditioning_in_weight-num_l...,4096,iw,500
3,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_4096-conditioning_iw-num_low_freq...,seed_2,eval-none,accuracy,"[0.533, 0.944, 0.946, 0.941, 0.937, 0.94000000...",dataset_size_4096-conditioning_in_weight-num_l...,4096,iw,500
4,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_4096-conditioning_iw-num_low_freq...,seed_2,eval-none,context contains query class,"[0.946, 0.946, 0.946, 0.946, 0.946, 0.946, 0.9...",dataset_size_4096-conditioning_in_weight-num_l...,4096,iw,500
...,...,...,...,...,...,...,...,...,...,...,...
1318,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_262144-conditioning_iw-num_low_fr...,seed_2,eval-iw,context contains query class,"[0.942, 0.942, 0.942, 0.942, 0.942, 0.942, 0.9...",dataset_size_262144-conditioning_in_weight-num...,262144,iw,500
1319,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_262144-conditioning_iw-num_low_fr...,seed_2,eval-iw,loss,"[0.7783243656158447, 0.05321662500500679, 0.05...",dataset_size_262144-conditioning_in_weight-num...,262144,iw,500
1320,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_262144-conditioning_iw-num_low_fr...,seed_2,pretraining,accuracy,"[0.502, 0.9902, 0.9902, 0.9902, 0.9902, 0.9902...",dataset_size_262144-conditioning_in_weight-num...,262144,iw,500
1321,transformer,binary_synthetic-transformer-num_low_freq,dataset_size_262144-conditioning_iw-num_low_fr...,seed_2,pretraining,context contains query class,"[0.9435, 0.9435, 0.9435, 0.9435, 0.9435, 0.943...",dataset_size_262144-conditioning_in_weight-num...,262144,iw,500


In [10]:
stats["eval_name"].unique()

array(['eval-none-flip_label', 'eval-none', 'eval-ic-flip_label',
       'eval-ic', 'eval-iw-flip_label', 'eval-iw', 'pretraining'],
      dtype=object)